# Find a way to downsample total contacts

In [1]:
import cooler
import numpy as np
import pandas as pd
import anndata
from pathlib import Path
from glob import glob
import xarray as xr

In [2]:
root = Path("/tscc/projects/ps-epigen/users/biy022/scmethylhic/human_hippocampus/snm3c/Combined/hic")
contact_files_root = Path("/tscc/projects/ps-epigen/users/biy022/scmethylhic/human_hippocampus/snm3c/")

In [3]:
# Load meta file
meta_file = "/tscc/projects/ps-renlab2/sel041/scmethylhic/human_hippocampus/concat/human_aging_final_metadata.csv.gz"

In [4]:
meta_table = pd.read_csv(meta_file, header=0, index_col=1)

In [5]:
meta_table.final_cluster.unique()

array(['Oligo', 'Astro2', 'VIP', 'CA', 'Micro2', 'Micro1', 'OPC', 'SUB',
       'Astro1', 'NR2F2-LAMP5', 'PVALB', 'SST', 'VLMC', 'Endo', 'DG'],
      dtype=object)

In [6]:
for cluster, sub_df in meta_table.groupby("final_cluster"):
    num_contacts = sub_df.TotalContacts.sum()
    print(f"{cluster}\t{num_contacts}")

Astro1	584817830
Astro2	530381153
CA	47299421
DG	142641439
Endo	35960315
Micro1	680222466
Micro2	498396209
NR2F2-LAMP5	71658080
OPC	508608514
Oligo	5923563206
PVALB	62595523
SST	84378427
SUB	522947240
VIP	128762144
VLMC	77588433


In [7]:
oligo_table = meta_table[meta_table.final_cluster == "Oligo"]

In [8]:
for age, sub_df in oligo_table.groupby("group"):
    num_contacts = sub_df.TotalContacts.sum()
    print(f"{age}\t{num_contacts}")

20-40	1392102328
40-60	1623471442
60-80	1938307926
80-100	969681510


## Test the sampling process

In [9]:
domain_dir = root / "domain_cluster_age"
oligo_dir = domain_dir / "Oligo"
test_age_dir = oligo_dir / "Age2040"

In [10]:
curr_cell_table = pd.read_csv(
    test_age_dir / "cell_table.csv", 
    names=["barcode", "path", "group"], 
    index_col=0
)

In [11]:
contact_dfs = []
for barcode in curr_cell_table.index:
    donor_name = barcode.split("-")[0]
    curr_contact_path = contact_files_root / f"{donor_name}_deep" / "hic" / "raw" / f"{barcode}.3C.contact.tsv.gz"
    curr_contact_df = pd.read_csv(
        curr_contact_path, 
        header=None, 
        index_col=None, 
        sep="\t",
        names=["frag1", "chr1", "pos1", "strand1", "frag2", "chr2", "pos2", "strand2"]
    )
    curr_contact_df["barcode"] = barcode
    contact_dfs.append(curr_contact_df)

In [12]:
curr_total_contact_df = pd.concat(contact_dfs, axis=0)

In [13]:
curr_total_contact_df.shape

(437498313, 9)

In [14]:
target_n = 250000000
curr_downsample_contact_df = curr_total_contact_df.sample(n=target_n, axis=0)

In [15]:
curr_downsample_contact_df.shape

(250000000, 9)

In [18]:
if not (oligo_dir / "Age2040_Downsample" / "contacts").exists():
    (oligo_dir / "Age2040_Downsample" / "contacts").mkdir(exist_ok=True, parents=True)
for barcode, sub_df in curr_downsample_contact_df.groupby("barcode"):
    sub_df.to_csv(
        oligo_dir / "Age2040_Downsample" / "contacts" / f"{barcode}.3C.contact.tsv.gz",
        header=False,
        index=False,
        compression="gzip",
        sep="\t"
    )

## Write a function for downsampling contact files

In [20]:
def down_sample_contact_files(
    input_cell_table_file,
    output_dir,
    target_number,
    sep,
):
    cell_table = pd.read_csv(
        input_cell_table_file,
        usecols=[0, 1],
        names=["barcode", "path"],
        sep=sep,
        index_col=0
    )

    contact_dfs = []
    for barcode in cell_table.index:
        donor_name = barcode.split("-")[0]
        curr_contact_path = contact_files_root / f"{donor_name}_deep" / "hic" / "rmbkl" / f"{barcode}.contact.rmbkl.tsv.gz"
        curr_contact_df = pd.read_csv(
            curr_contact_path, 
            header=None, 
            index_col=None, 
            sep="\t",
            names=["frag1", "chr1", "pos1", "strand1", "frag2", "chr2", "pos2", "strand2"]
        )
        curr_contact_df["barcode"] = barcode
        contact_dfs.append(curr_contact_df)
    curr_total_contact_df = pd.concat(contact_dfs, axis=0)
    curr_downsample_contact_df = curr_total_contact_df.sample(n=target_number, axis=0)

    if not output_dir.exists():
        output_dir.mkdir(exist_ok=True, parents=True)
    for barcode, sub_df in curr_downsample_contact_df.groupby("barcode"):
        sub_df.to_csv(
            output_dir / f"{barcode}.contact.downsample.rmbkl.tsv.gz",
            header=False,
            index=False,
            compression="gzip",
            sep="\t"
        )
    

In [21]:
def get_target_number(input_cell_table_file, sep):
    
    cell_table = pd.read_csv(
        input_cell_table_file,
        usecols=[0, 1],
        names=["barcode", "path"],
        sep=sep,
        index_col=0
    )

    contact_dfs = []
    for barcode in cell_table.index:
        donor_name = barcode.split("-")[0]
        curr_contact_path = contact_files_root / f"{donor_name}_deep" / "hic" / "rmbkl" / f"{barcode}.contact.rmbkl.tsv.gz"
        curr_contact_df = pd.read_csv(
            curr_contact_path, 
            header=None, 
            index_col=None, 
            sep="\t",
            names=["frag1", "chr1", "pos1", "strand1", "frag2", "chr2", "pos2", "strand2"]
        )
        curr_contact_df["barcode"] = barcode
        contact_dfs.append(curr_contact_df)
    curr_total_contact_df = pd.concat(contact_dfs, axis=0)

    return curr_total_contact_df.shape[0]

In [22]:
oligo_age_80100_cell_table = domain_dir / "Oligo" / "Age80100" / "cell_table.csv"
sep = ","
oligo_target_number = get_target_number(oligo_age_80100_cell_table, sep=",")

In [23]:
oligo_target_number

253689700

In [25]:
down_sample_contact_files(
    domain_dir / "Oligo" / "Age2040" / "cell_table.csv",
    domain_dir / "Oligo" / "Age2040_DOWN" / "contacts",
    oligo_target_number,
    ","
)

In [27]:
down_sample_contact_files(
    domain_dir / "Oligo" / "Age4060" / "cell_table.csv",
    domain_dir / "Oligo" / "Age4060_DOWN" / "contacts",
    oligo_target_number,
    ","
)

In [28]:
down_sample_contact_files(
    domain_dir / "Oligo" / "Age6080" / "cell_table.csv",
    domain_dir / "Oligo" / "Age6080_DOWN" / "contacts",
    oligo_target_number,
    ","
)

## Create domain calling scripts

In [2]:
import schicluster
import os

In [3]:
PKG_DIR = schicluster.__path__[0]

In [4]:
# Load meta file
meta_file = "/tscc/projects/ps-renlab2/sel041/scmethylhic/human_hippocampus/concat/human_aging_final_metadata.csv.gz"

In [5]:
def prep_dir(output_dir, chunk_df, template, params):
    os.makedirs(output_dir, exist_ok=True)
    cell_table_path = os.path.join(output_dir, "cell_table.csv")
    chunk_df.to_csv(cell_table_path, header=False, index=True)
    params_str = "\n".join(f"{k} = {v}" for k, v in params.items())

    with open(os.path.join(output_dir, "Snakefile_master"), "w") as f:
        f.write(params_str + template)
    return

In [6]:
sbatch_header = (
    "#! /bin/bash\n"
    "#SBATCH -p condo\n"
    "#SBATCH -q condo\n"
    "#SBATCH -A csd772\n"
    "#SBATCH -J hh_{0}_s{1}_{2}\n"
    "#SBATCH -N {3}\n"
    "#SBATCH -c {4}\n"
    "#SBATCH --mem {5}G\n"
    "#SBATCH -t {6}:00:00\n"
    "#SBATCH -o /tscc/projects/ps-epigen/users/biy022/scmethylhic/human_hippocampus/snm3c/Combined/hic/{0}/domain_scripts/step{1}_{2}.out\n"
    "#SBATCH -e /tscc/projects/ps-epigen/users/biy022/scmethylhic/human_hippocampus/snm3c/Combined/hic/{0}/domain_scripts/step{1}_{2}.err\n"
    "#SBATCH --mail-user biy022@health.ucsd.edu\n"
    "#SBATCH --mail-type FAIL\n"
    "\n"
    "source ~/.bashrc\n"
    "conda activate schicluster\n"
    "cd {7}\n"
)

In [7]:
root = Path("/tscc/projects/ps-epigen/users/biy022/scmethylhic/human_hippocampus/snm3c/Combined/hic")
parent_root = Path("/tscc/projects/ps-epigen/users/biy022/scmethylhic")
chrom_size_path = parent_root / "genome" / "genome_hg38" / "hg38.autosomal.chrom.sizes"

### Test with Oligo Age2040

In [4]:
domain_cluster_age_dir = root / "domain_cluster_age"
oligo_dir = domain_cluster_age_dir / "Oligo"

In [5]:
oligo_2040_dir = oligo_dir / "Age2040_DOWN"
cool_list = list(oligo_2040_dir.glob("impute/25kb/chunk*/*cool"))

In [6]:
cell_table = pd.DataFrame(
    cool_list,
    index=[xx.stem for xx in cool_list],
    columns=["cool_path"]
)
# cell_table.to_csv(oligo_2040_dir / "cell_table_bulk.csv", header=False, index=True)

In [11]:
params = {
    "resolution": 25000,
    "chrom_size_path": '"{}"'.format(chrom_size_path),
}

In [12]:
chunksize = 200
resolution = 25000

In [13]:
with open("{}/cool/Snakefile_chunk_template".format(PKG_DIR)) as f:
    GENERATE_MATRIX_CHUNK_TEMPLATE = f.read()
(oligo_2040_dir / "Age2040").mkdir(exist_ok=True, parents=True)

total_chunk_dirs = []
if cell_table.shape[0] <= chunksize:
    curr_dir = oligo_2040_dir / ("Age2040" + "_chunk0")
    params["cell_table_path"] = '"{}"'.format(curr_dir / "cell_table.csv")
    prep_dir(str(curr_dir), cell_table, GENERATE_MATRIX_CHUNK_TEMPLATE, params)
    total_chunk_dirs.append(curr_dir)
else:
    cell_table["chunk"] = [i // chunksize for i in range(0, cell_table.shape[0])]
    for chunk, chunk_df in cell_table.groupby("chunk"):
        curr_dir = oligo_2040_dir / ("Age2040" + f"_chunk{chunk}")
        params["cell_table_path"] = '"{}"'.format(curr_dir / "cell_table.csv")
        prep_dir(str(curr_dir), chunk_df, GENERATE_MATRIX_CHUNK_TEMPLATE, params)
        total_chunk_dirs.append(curr_dir)

In [14]:
script_dir = oligo_2040_dir / "domain_scripts"
script_dir.mkdir(exist_ok=True)
with (script_dir / "snakemake_cmd_step1.txt").open("w") as f:
    for chunk_dir in total_chunk_dirs:
        cmd = "snakemake -d {0} --snakefile {0}/Snakefile_master -j 5 --rerun-incomplete".format(chunk_dir)
        f.write(cmd + "\n")

In [15]:
if "cell_table_path" in params.keys():
    params.pop("cell_table_path")
params["output_dir"] = '"{}"'.format(oligo_2040_dir)
params_str = "\n".join("{} = {}".format(k, v) for k, v in params.items())

with open("{}/cool/Snakefile_group_template".format(PKG_DIR), "r") as f:
    GENERATE_MATRIX_GROUP_TEMPLATE = f.read()

with open(oligo_2040_dir / "domain_scripts" / "Snakefile", "w") as f:
    f.write(params_str + "\n" + GENERATE_MATRIX_GROUP_TEMPLATE)

with open(oligo_2040_dir / "domain_scripts" / "snakemake_cmd_step2.txt", "w") as f:
    cmd = "snakemake -d {} --snakefile {} -j 10 --rerun-incomplete".format(
        oligo_2040_dir,
        oligo_2040_dir / "domain_scripts" / "Snakefile"
    )
    f.write(cmd + "\n")

In [16]:
chunksize = 40
with open(oligo_2040_dir / "domain_scripts" / "snakemake_cmd_step1.txt", "r") as finput:
    cmds = finput.readlines()

index = 0
for i in range(0, len(cmds), chunksize):
    with open(oligo_2040_dir / "domain_scripts" / "snakemake_cmd_step1_{}.txt".format(index), "w") as f:
        f.write("\n".join([xx.strip() for xx in cmds[i:(i+chunksize)]]) + "\n")
    with open(oligo_2040_dir / "domain_scripts" / "step1_{}.sbatch".format(index), "w") as f:
        f.write(sbatch_header.format("domain_cluster_age/Oligo/Age2040_DOWN", 1, index, 1, 16, 100, 4, oligo_2040_dir / "domain_scripts") + "\n")
        f.write("bash snakemake_cmd_step1_{}.txt\n".format(index))
    index += 1

### Test with Oligo Age4060

In [17]:
domain_cluster_age_dir = root / "domain_cluster_age"
oligo_dir = domain_cluster_age_dir / "Oligo"

In [18]:
oligo_4060_dir = oligo_dir / "Age4060_DOWN"
cool_list = list(oligo_4060_dir.glob("impute/25kb/chunk*/*cool"))

In [19]:
cell_table = pd.DataFrame(
    cool_list,
    index=[xx.stem for xx in cool_list],
    columns=["cool_path"]
)
cell_table.to_csv(oligo_4060_dir / "cell_table.csv", header=False, index=True)

In [20]:
params = {
    "resolution": 25000,
    "chrom_size_path": '"{}"'.format(chrom_size_path),
}

In [21]:
chunksize = 200
resolution = 25000

In [22]:
with open("{}/cool/Snakefile_chunk_template".format(PKG_DIR)) as f:
    GENERATE_MATRIX_CHUNK_TEMPLATE = f.read()
(oligo_4060_dir / "Age4060").mkdir(exist_ok=True, parents=True)

total_chunk_dirs = []
if cell_table.shape[0] <= chunksize:
    curr_dir = oligo_4060_dir / ("Age4060" + "_chunk0")
    params["cell_table_path"] = '"{}"'.format(curr_dir / "cell_table.csv")
    prep_dir(str(curr_dir), cell_table, GENERATE_MATRIX_CHUNK_TEMPLATE, params)
    total_chunk_dirs.append(curr_dir)
else:
    cell_table["chunk"] = [i // chunksize for i in range(0, cell_table.shape[0])]
    for chunk, chunk_df in cell_table.groupby("chunk"):
        curr_dir = oligo_4060_dir / ("Age4060" + f"_chunk{chunk}")
        params["cell_table_path"] = '"{}"'.format(curr_dir / "cell_table.csv")
        prep_dir(str(curr_dir), chunk_df, GENERATE_MATRIX_CHUNK_TEMPLATE, params)
        total_chunk_dirs.append(curr_dir)

In [23]:
script_dir = oligo_4060_dir / "domain_scripts"
script_dir.mkdir(exist_ok=True)
with (script_dir / "snakemake_cmd_step1.txt").open("w") as f:
    for chunk_dir in total_chunk_dirs:
        cmd = "snakemake -d {0} --snakefile {0}/Snakefile_master -j 5 --rerun-incomplete".format(chunk_dir)
        f.write(cmd + "\n")

In [24]:
if "cell_table_path" in params.keys():
    params.pop("cell_table_path")
params["output_dir"] = '"{}"'.format(oligo_4060_dir)
params_str = "\n".join("{} = {}".format(k, v) for k, v in params.items())

with open("{}/cool/Snakefile_group_template".format(PKG_DIR), "r") as f:
    GENERATE_MATRIX_GROUP_TEMPLATE = f.read()

with open(oligo_4060_dir / "domain_scripts" / "Snakefile", "w") as f:
    f.write(params_str + "\n" + GENERATE_MATRIX_GROUP_TEMPLATE)

with open(oligo_4060_dir / "domain_scripts" / "snakemake_cmd_step2.txt", "w") as f:
    cmd = "snakemake -d {} --snakefile {} -j 10 --rerun-incomplete".format(
        oligo_4060_dir,
        oligo_4060_dir / "domain_scripts" / "Snakefile"
    )
    f.write(cmd + "\n")

In [25]:
chunksize = 40
with open(oligo_4060_dir / "domain_scripts" / "snakemake_cmd_step1.txt", "r") as finput:
    cmds = finput.readlines()

index = 0
for i in range(0, len(cmds), chunksize):
    with open(oligo_4060_dir / "domain_scripts" / "snakemake_cmd_step1_{}.txt".format(index), "w") as f:
        f.write("\n".join([xx.strip() for xx in cmds[i:(i+chunksize)]]) + "\n")
    with open(oligo_4060_dir / "domain_scripts" / "step1_{}.sbatch".format(index), "w") as f:
        f.write(sbatch_header.format("domain_cluster_age/Oligo/Age4060_DOWN", 1, index, 1, 16, 100, 4, oligo_4060_dir / "domain_scripts") + "\n")
        f.write("bash snakemake_cmd_step1_{}.txt\n".format(index))
    index += 1

### Test with Oligo Age6080

In [26]:
domain_cluster_age_dir = root / "domain_cluster_age"
oligo_dir = domain_cluster_age_dir / "Oligo"

In [27]:
oligo_6080_dir = oligo_dir / "Age6080_DOWN"
cool_list = list(oligo_6080_dir.glob("impute/25kb/chunk*/*cool"))

In [28]:
cell_table = pd.DataFrame(
    cool_list,
    index=[xx.stem for xx in cool_list],
    columns=["cool_path"]
)
cell_table.to_csv(oligo_6080_dir / "cell_table.csv", header=False, index=True)

In [29]:
params = {
    "resolution": 25000,
    "chrom_size_path": '"{}"'.format(chrom_size_path),
}

In [30]:
chunksize = 200
resolution = 25000

In [31]:
with open("{}/cool/Snakefile_chunk_template".format(PKG_DIR)) as f:
    GENERATE_MATRIX_CHUNK_TEMPLATE = f.read()
(oligo_6080_dir / "Age6080").mkdir(exist_ok=True, parents=True)

total_chunk_dirs = []
if cell_table.shape[0] <= chunksize:
    curr_dir = oligo_6080_dir / ("Age6080" + "_chunk0")
    params["cell_table_path"] = '"{}"'.format(curr_dir / "cell_table.csv")
    prep_dir(str(curr_dir), cell_table, GENERATE_MATRIX_CHUNK_TEMPLATE, params)
    total_chunk_dirs.append(curr_dir)
else:
    cell_table["chunk"] = [i // chunksize for i in range(0, cell_table.shape[0])]
    for chunk, chunk_df in cell_table.groupby("chunk"):
        curr_dir = oligo_6080_dir / ("Age6080" + f"_chunk{chunk}")
        params["cell_table_path"] = '"{}"'.format(curr_dir / "cell_table.csv")
        prep_dir(str(curr_dir), chunk_df, GENERATE_MATRIX_CHUNK_TEMPLATE, params)
        total_chunk_dirs.append(curr_dir)

In [32]:
script_dir = oligo_6080_dir / "domain_scripts"
script_dir.mkdir(exist_ok=True)
with (script_dir / "snakemake_cmd_step1.txt").open("w") as f:
    for chunk_dir in total_chunk_dirs:
        cmd = "snakemake -d {0} --snakefile {0}/Snakefile_master -j 5 --rerun-incomplete".format(chunk_dir)
        f.write(cmd + "\n")

In [33]:
if "cell_table_path" in params.keys():
    params.pop("cell_table_path")
params["output_dir"] = '"{}"'.format(oligo_6080_dir)
params_str = "\n".join("{} = {}".format(k, v) for k, v in params.items())

with open("{}/cool/Snakefile_group_template".format(PKG_DIR), "r") as f:
    GENERATE_MATRIX_GROUP_TEMPLATE = f.read()

with open(oligo_6080_dir / "domain_scripts" / "Snakefile", "w") as f:
    f.write(params_str + "\n" + GENERATE_MATRIX_GROUP_TEMPLATE)

with open(oligo_6080_dir / "domain_scripts" / "snakemake_cmd_step2.txt", "w") as f:
    cmd = "snakemake -d {} --snakefile {} -j 10 --rerun-incomplete".format(
        oligo_6080_dir,
        oligo_6080_dir / "domain_scripts" / "Snakefile"
    )
    f.write(cmd + "\n")

In [34]:
chunksize = 40
with open(oligo_6080_dir / "domain_scripts" / "snakemake_cmd_step1.txt", "r") as finput:
    cmds = finput.readlines()

index = 0
for i in range(0, len(cmds), chunksize):
    with open(oligo_6080_dir / "domain_scripts" / "snakemake_cmd_step1_{}.txt".format(index), "w") as f:
        f.write("\n".join([xx.strip() for xx in cmds[i:(i+chunksize)]]) + "\n")
    with open(oligo_6080_dir / "domain_scripts" / "step1_{}.sbatch".format(index), "w") as f:
        f.write(sbatch_header.format("domain_cluster_age/Oligo/Age6080_DOWN", 1, index, 1, 16, 100, 4, oligo_6080_dir / "domain_scripts") + "\n")
        f.write("bash snakemake_cmd_step1_{}.txt\n".format(index))
    index += 1